<h4>-----------------------------------------------------------------------------<br>Copyright (c) 2022, Lucid Vision Labs, Inc.</h4>
<h5> THE  SOFTWARE  IS  PROVIDED  "AS IS",  WITHOUT  WARRANTY  OF  ANY  KIND,<br>EXPRESS  OR  IMPLIED,  INCLUDING  BUT  NOT  LIMITED  TO  THE  WARRANTIES<br>OF  MERCHANTABILITY,  FITNESS  FOR  A  PARTICULAR  PURPOSE  AND<br>NONINFRINGEMENT.  IN  NO  EVENT  SHALL  THE  AUTHORS  OR  COPYRIGHT  HOLDERS<br>BE  LIABLE  FOR  ANY  CLAIM,  DAMAGES  OR  OTHER  LIABILITY,  WHETHER  IN  AN<br>ACTION  OF  CONTRACT,  TORT  OR  OTHERWISE,  ARISING  FROM,  OUT  OF  OR  IN<br>CONNECTION  WITH  THE  SOFTWARE  OR  THE  USE  OR  OTHER  DEALINGS  IN <br> THE  SOFTWARE.<br>-----------------------------------------------------------------------------</h5>

# Version 1.0

# Barcode example
### This application will use <a href="https://pypi.org/project/pyzbar/">pyzbar</a> to decode barcodes from an image and <a href="https://pypi.org/project/opencv-python/">opencv-python</a> to display the image on screen.

# Cameras

- Phoenix
- Triton
- Atlas
- Atlas10

# Dependencies

This application requires the following dependencies:
- opencv-python 4.5.3.56
- numpy 1.21.6
- pyzbar 0.1.9

In [1]:
!pip install opencv-python==4.5.3.56
!pip install numpy===1.21.6
!pip install pyzbar==0.1.9

# Testing

This application has been tested with Python v3.7.11

In [2]:
from arena_api import enums
from arena_api.buffer import BufferFactory
from arena_api.system import system

In [3]:
import cv2 
import numpy as np
from pyzbar import pyzbar
from pyzbar.pyzbar import ZBarSymbol

#### Set **use_camera_image = True** to use a camera image

#### Set **use_camera_image = False** to use a predefined image (sample_barcodes.png in this directory):
<img src="sample_barcodes.png" width=400 height=400 />

In [4]:
use_camera_image = True

### The variable **zsymbols** explicitly defines which barcode types to look for, which can be helpful to prevent false readings.

### See the <a href="https://github.com/NaturalHistoryMuseum/pyzbar/blob/master/pyzbar/wrapper.py">ZBarSymbol enumerations</a> for a list of symbols.

In [5]:
zsymbols = [ZBarSymbol.I25, 
            ZBarSymbol.CODE128, 
            ZBarSymbol.EAN8, 
            ZBarSymbol.EAN13, 
            ZBarSymbol.QRCODE, 
            ZBarSymbol.UPCE]

### The following variables define the display window width and height

In [6]:
window_width = 800
window_height = 600

### The following function waits for the user to select a device

In [7]:
def select_device_from_user_input():
    
    device_infos = None
    selected_index = None
    while selected_index is None:
        device_infos = system.device_infos
        if len(device_infos) == 0:
            print("No camera connected\nPress enter to search again")
            input()
            continue
        print("Devices found:")
        for i in range(len(device_infos)):
            print(f"\t{i}. {device_infos[i]['model']} SN: {device_infos[i]['serial']}")

        while True:
            line = input("Selection: ")
            try:
                selected_index = int(line)
                if 0 <= selected_index < len(device_infos):
                    break
                else:
                    print(f"Please enter a valid number between 0 and {len(device_infos)-1}\n")
            except Exception as e:
                print("\nPlease enter a valid number\n")

    selected_model = device_infos[selected_index]['model']
    print(f"\nCreate device: {selected_model}...")
    device = system.create_device(device_infos=device_infos[selected_index])[0]
    
    return device

### The following function sets up and runs the camera

In [8]:
def run_camera():

    # create devices
    device = select_device_from_user_input()
    
    device.tl_stream_nodemap.get_node('StreamBufferHandlingMode').value = 'NewestOnly'
    device.tl_stream_nodemap.get_node('StreamPacketResendEnable').value = True
    device.tl_stream_nodemap.get_node('StreamAutoNegotiatePacketSize').value = True
    
    isp_bayer_pattern = device.nodemap.get_node('IspBayerPattern').value
    is_color_camera = False
    
    if isp_bayer_pattern != 'NONE':
        is_color_camera = True
    
    if is_color_camera == True:
        device.nodemap.get_node('PixelFormat').value = "BayerRG8"
    else:
        device.nodemap.get_node('PixelFormat').value = "Mono8"
        
    # grab images -------------------------------------------------------------

    get_image_buffers(device, is_color_camera)

    # clean up ----------------------------------------------------------------

    # This function call with no arguments will destroy all of the
    # created devices. Having this call here is optional, if it is not
    # here it will be called automatically when the system module is unloading.
    system.destroy_device()
    print('Destroyed all created devices')

In [9]:
def read_barcodes(input_img, display_img):    
    barcodes = pyzbar.decode(input_img, zsymbols)
    
    num_barcodes = 1
    
    for barcode in barcodes:
        # extract the bounding box location of the barcode and draw the
        # bounding box surrounding the barcode on the image
        (x, y, w, h) = barcode.rect
        
        if num_barcodes == 1:
            cv2.rectangle(display_img, (x, y), (x + w, y + h), (0, 0, 255), 2)
        elif num_barcodes == 2:
            cv2.rectangle(display_img, (x, y), (x + w, y + h), (0, 255, 0), 2)
        elif num_barcodes == 3:
            cv2.rectangle(display_img, (x, y), (x + w, y + h), (255, 0, 0), 2)
        
        # the barcode data is a bytes object so if we want to draw it on
        # our output image we need to convert it to a string first
        barcodeData = barcode.data.decode("utf-8")
        barcodeType = barcode.type
        
        # draw the barcode data and barcode type on the image
        text = "{} ({})".format(barcodeData, barcodeType)
        
        if num_barcodes == 1:
            cv2.putText(display_img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif num_barcodes == 2:
            cv2.putText(display_img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        elif num_barcodes == 3:
            cv2.putText(display_img, text, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        
        # print the barcode type and data to the terminal
        #print("[INFO] Found {} barcode: {}".format(barcodeType, barcodeData))
        
        num_barcodes = num_barcodes + 1
        if num_barcodes > 3:
            num_barcodes = 1
    
    return display_img

In [10]:
def get_image_buffers(device, is_color_camera=False):
    key = -1
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
        
    device.start_stream()
    
    while True:    
        image_buffer = device.get_buffer()  # optional args
        nparray = np.ctypeslib.as_array(image_buffer.pdata,shape=(image_buffer.height, image_buffer.width, int(image_buffer.bits_per_pixel / 8))).reshape(image_buffer.height, image_buffer.width, int(image_buffer.bits_per_pixel / 8))
        
        if is_color_camera == True:
            #converted_img = BufferFactory.convert(image_buffer, enums.PixelFormat.BGR8)
            #display_img = np.ctypeslib.as_array(converted_img.pdata,shape=(converted_img.height, converted_img.width, int(converted_img.bits_per_pixel / 8))).reshape(converted_img.height, converted_img.width, int(converted_img.bits_per_pixel / 8))
            display_img = cv2.cvtColor(nparray, cv2.COLOR_BayerBG2BGR)
            nparray = cv2.cvtColor(display_img, cv2.COLOR_BGR2GRAY)
        else:
            display_img = cv2.cvtColor(nparray, cv2.COLOR_GRAY2BGR)
        
        decoded_img = read_barcodes(nparray, display_img)
        
        cv2.resizeWindow("Image", window_width, window_height)
        cv2.imshow("Image", decoded_img)
        
        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break

        #BufferFactory.destroy(converted_image)
        device.requeue_buffer(image_buffer)    
        
    cv2.destroyAllWindows()
    device.stop_stream()

In [11]:
def read_sample_image():
    
    key = -1
    cv2.namedWindow("Image", cv2.WINDOW_NORMAL)
    
    nparray = cv2.imread("sample_barcodes.png", cv2.IMREAD_GRAYSCALE)
    display_img = cv2.cvtColor(nparray, cv2.COLOR_GRAY2BGR)
    
    decoded_img = read_barcodes(nparray, display_img)
    
    while True:
        cv2.resizeWindow("Image", window_width, window_height)
        cv2.imshow("Image", decoded_img)

        key = cv2.waitKey(1) & 0xFF
        if key == ord("q"):
            break
            
    cv2.destroyAllWindows()

In [12]:
if __name__ == '__main__':
    try:
        print('WARNING:\nTHIS EXAMPLE MIGHT CHANGE THE DEVICE(S) SETTINGS!')
        print('Example started')
        
        if use_camera_image == True:
            run_camera()
        else:
            read_sample_image()
        
        print('Example finished successfully')
    except BaseException as be:
        print(be)
        raise be

THIS EXAMPLE MIGHT CHANGE THE DEVICE(S) SETTINGS!
Example started
Example finished successfully
